In [32]:
import os

import pandas as pd
import numpy as np

In [2]:
DATA_DIR = os.path.join('..', 'data')
os.listdir(DATA_DIR)

['history.csv', 'submission.csv', 'employees.csv']

In [3]:
hist = pd.read_csv(os.path.join(DATA_DIR, 'history.csv'))
empls = pd.read_csv(os.path.join(DATA_DIR, 'employees.csv'))
sbmsn = pd.read_csv(os.path.join(DATA_DIR, 'submission.csv'))

In [4]:
hist.head()

,EmployeeID,Date,DevCenterID,SBUID,PositionID,PositionLevel,IsTrainee,LanguageLevelID,CustomerID,ProjectID,...,HourLockedReserve,OnSite,CompetenceGroupID,FunctionalOfficeID,PaymentTypeId,BonusOneTime,APM,WageGross,MonthOnPosition,MonthOnSalary
0,00116D71-E87D-4B64-A566-1F29B2A798A8,7/1/17,3,292,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,...,0,0,15,1,9,0,39,0.887446,1,1
1,00116D71-E87D-4B64-A566-1F29B2A798A8,8/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,...,0,0,15,1,9,200,28,0.887446,2,2
2,00116D71-E87D-4B64-A566-1F29B2A798A8,9/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,...,0,0,15,1,9,0,43,0.887446,3,3
3,00116D71-E87D-4B64-A566-1F29B2A798A8,10/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,...,0,0,15,1,9,0,49,0.887446,4,4
4,00116D71-E87D-4B64-A566-1F29B2A798A8,11/1/17,3,332,70,2,0,11,893EA22F-08BE-4F11-AD93-C50746E4565F,7F97465B-ED3C-45DB-BE96-86C7E493F0CD,...,0,0,15,1,9,0,43,0.887446,5,5


In [5]:
empls.head()

,EmployeeID,HiringDate,DismissalDate
0,01EDC1A7-5DA7-4939-854B-0002E0DF548D,9/1/18,NaN
1,91C351D9-38A1-40CB-9B69-000774211859,1/1/18,2/1/19
2,D699327D-8A78-4FBE-9CEA-001355717D3F,2/1/18,NaN
3,22102E27-5EEA-44F9-8632-0017B04B1236,10/1/14,5/1/18
4,08C0AFA9-ABC3-4FAF-8DE2-0018AE9A69F4,9/1/10,NaN


In [6]:
df = hist.merge(empls)

In [8]:
len(df)

87766

In [9]:
len(hist)

87766

In [22]:
len(df[df.EmployeeID == df.EmployeeID.unique()[0]]['DismissalDate'])

20

In [23]:
[1,2,3] + [4,5,6]

[1, 2, 3, 4, 5, 6]

In [30]:
def label_df(df, n_month=3):
    labels = []
    for emp in df.EmployeeID.unique():
        curr_emp = list(df[df.EmployeeID == emp]['DismissalDate'])
        len_emp = len(curr_emp)
        if pd.isnull(curr_emp[0]):
            labels += [0 for i in range(len_emp - n_month)] + [2 for i in range(n_month)]
        else:
            labels += [0 for i in range(len_emp - n_month)] + [1 for i in range(n_month)]
    return labels

In [31]:
lbls= label_df(df)

In [33]:
np.unique(lbls)

array([0, 1, 2])

In [36]:
unique, counts = np.unique(lbls, return_counts=True)
dict(zip(unique, counts))

{0: 71647, 1: 3051, 2: 13068}

In [37]:
df['target'] = lbls

In [50]:
np.unique(df.target)

array([0, 1, 2])

In [49]:
max(list(df.groupby('EmployeeID').count()['target']))

20

In [51]:
df = df[df.target != 2]

In [52]:
np.unique(df.target)

array([0, 1])

In [54]:
df.IsTrainee

0        0
1        0
2        0
3        0
4        0
        ..
87758    1
87759    1
87760    1
87761    1
87762    1
Name: IsTrainee, Length: 74698, dtype: int64

In [58]:
df[df.EmployeeID == df.EmployeeID.unique()[0]].tail(1)

,EmployeeID,Date,DevCenterID,SBUID,PositionID,PositionLevel,IsTrainee,LanguageLevelID,CustomerID,ProjectID,...,FunctionalOfficeID,PaymentTypeId,BonusOneTime,APM,WageGross,MonthOnPosition,MonthOnSalary,HiringDate,DismissalDate,target
16,00116D71-E87D-4B64-A566-1F29B2A798A8,11/1/18,3,284,70,2,0,11,888A91D6-1F35-4337-ADCD-E4D7F0399DF1,4AAF7425-575B-46CB-A42D-F43A53187D32,...,1,22,0,44,0.988548,17,3,6/1/13,NaN,0


In [59]:
def get_df_for_pred(df, needed):
    df_to_pred = pd.DataFrame()
    for emp in df.EmployeeID.unique():
        df_to_pred.append(df[df.EmployeeID == emp].tail(1), ignore_index=True)
    return df_to_pred

In [61]:
df_to = get_df_for_pred(df)

In [62]:
df_to

""


In [66]:
wdf = hist[hist['EmployeeID'] == 'FFF3B179-1D20-40FF-A330-A051BDF37301']

In [69]:
list(wdf.index)

[87756, 87757, 87758, 87759, 87760, 87761, 87762, 87763, 87764, 87765]